In [2]:
pip install tqdm


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import win32com.client
import getpass
import csv
from tqdm import tqdm

# ─── Config ────────────────────────────────────────────────────────────────
server = "APP03/HC-SC/GC/CA"
filepath = "csb\\imsd\\hcdir3.nsf"
view_name = "English / Anglais\\2. Employees, alphabetically"
output_csv = "hcl_export.csv"

# ─── Prompt for Notes password ─────────────────────────────────────────────
password = getpass.getpass("Enter your Notes password: ")

# ─── Connect to Notes ──────────────────────────────────────────────────────
session = win32com.client.Dispatch("Lotus.NotesSession")
session.Initialize(password)

db = session.GetDatabase(server, filepath)
if not db.IsOpen:
    db.Open()

view = db.GetView(view_name)
doc = view.GetFirstDocument()

# ─── Extract all documents with progress (no denominator) ──────────────────
print("📊 Exporting all people with all fields...\n")
all_field_names = set()
documents = []

pbar = tqdm(desc="Exporting", unit="doc", dynamic_ncols=True, total=0)
while doc is not None:
    row = {}
    for item in doc.Items:
        try:
            name = item.Name
            value = item.Values
            if isinstance(value, list):
                value = ", ".join(str(v) for v in value)
            else:
                value = str(value)
            row[name] = value
            all_field_names.add(name)
        except Exception:
            pass  # skip unreadable items
    documents.append(row)
    doc = view.GetNextDocument(doc)
    pbar.update()
pbar.close()

all_field_names = sorted(all_field_names)
print(f"\n✅ Processed {len(documents)} documents with {len(all_field_names)} unique fields.")

# ─── Write to CSV ──────────────────────────────────────────────────────────
print(f"📤 Writing to file: {output_csv}")
with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=all_field_names, extrasaction='ignore')
    writer.writeheader()
    for row in documents:
        writer.writerow(row)

print("✅ Full export complete.")

📊 Exporting all people with all fields...



Exporting: 10622doc [18:05,  9.79doc/s]



✅ Processed 10622 documents with 264 unique fields.
📤 Writing to file: hcl_export.csv
✅ Full export complete.
